## NN Prediction

In [1]:
%load_ext autoreload
%autoreload 2|

In [3]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

import torch
import torch.nn as nn
from classifier.resnet18 import Resnet18Model
from data_loader.orb_features_data_module import ORBFeaturesDataModule
import pytorch_lightning as pl

In [4]:
model = Resnet18Model(nn.Sequential(nn.Linear(512, 512),
                                           nn.Linear(512, 512),
                                           nn.Linear(512, 2)
                                          ),
                             freeze_backbone=False
                            )

Using cache found in /home/alena/.cache/torch/hub/pytorch_vision_v0.10.0


In [8]:
dataset_module_params = {'root': "/home/alena/Documents/omni-vision/dataset", 
                  'dataset': ["dataset_2_cam_0",
                              "dataset_2_cam_1",
                              "dataset_2_cam_2",
                              "dataset_2_cam_3",
                              "dataset_2_cam_5",
                              "dataset_3_cam_0",
                              "dataset_3_cam_1",
                              "dataset_3_cam_2",
                              "dataset_3_cam_3",
                              "dataset_3_cam_5"
                             ],
                  'classification_threshold': 300,
                  'transform': model.transform,
                  'batch_size': 128+32,
                  'shuffle': True,
                  'num_workers': 6,
                  'num_val': 1000,
                  "num_test": 1000,}

datamodule = ORBFeaturesDataModule(**dataset_module_params)

In [ ]:
trainer = pl.Trainer(gpus=1, log_every_n_steps=1, max_epochs=8)
trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


datasetname: dataset_2_cam_0
num images: 2546
num labels: 2546
datasetname: dataset_2_cam_1
num images: 2546
num labels: 2546
datasetname: dataset_2_cam_2
num images: 2546
num labels: 2546
datasetname: dataset_2_cam_3
num images: 2546
num labels: 2546
datasetname: dataset_2_cam_5
num images: 2546
num labels: 2546
datasetname: dataset_3_cam_0
num images: 6376
num labels: 6376
datasetname: dataset_3_cam_1
num images: 6376
num labels: 6376
datasetname: dataset_3_cam_2
num images: 6376
num labels: 6376
datasetname: dataset_3_cam_3
num images: 6376
num labels: 6376
datasetname: dataset_3_cam_5


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


num images: 6376
num labels: 6376


Missing logger folder: /home/alena/repos/my_orb/orb-classification/notebooks/lightning_logs

  | Name       | Type       | Params
------------------------------------------
0 | activation | LogSoftmax | 0     
1 | loss       | NLLLoss    | 0     
2 | backbone   | ResNet     | 11.2 M
3 | fc         | Sequential | 526 K 
------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.811    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/alena/miniconda3/envs/omni-vision/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:659: UserWarning: Your `val_dataloader` has `shuffle=True`, it is strongly recommended that you turn this off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [7]:
from torchvision.io import read_image
import pandas as pd
from pathlib import Path
import os
import numpy as np

from data_loader.orb_features_dataset import ORBFeaturesDataset
from tqdm.auto import tqdm

In [7]:
dataset_params = {'root': "/home/alena/Documents/omni-vision/dataset", 
                  'datasets': [
#                               "dataset_1_cam_0",
                              "dataset_1_cam_1",
                              "dataset_1_cam_2",
                              "dataset_1_cam_3",
                              "dataset_1_cam_5"
                             ],
                  'classification_threshold': 300,
                  'transform': model.transform,
                  'combine_data': False
                 }

dataset = ORBFeaturesDataset(**dataset_params)

datasetname: dataset_2_cam_0
num images: 2546
num labels: 2546
datasetname: dataset_2_cam_1
num images: 2546
num labels: 2546
datasetname: dataset_2_cam_2
num images: 2546
num labels: 2546


In [8]:
dataset.images_list[0]

array(['/home/alena/Documents/omni-vision/dataset/dataset_2_cam_0/images/000000.png',
       '/home/alena/Documents/omni-vision/dataset/dataset_2_cam_0/images/000001.png',
       '/home/alena/Documents/omni-vision/dataset/dataset_2_cam_0/images/000002.png',
       ...,
       '/home/alena/Documents/omni-vision/dataset/dataset_2_cam_0/images/002543.png',
       '/home/alena/Documents/omni-vision/dataset/dataset_2_cam_0/images/002544.png',
       '/home/alena/Documents/omni-vision/dataset/dataset_2_cam_0/images/002545.png'],
      dtype='<U75')

In [9]:
def read_dataset_img(img_path, tranform):
    image = read_image(img_path)[None, ...]
    return tranform(image)

In [10]:
model = model.cpu()
model = model.eval()

In [11]:
predictions = []

for d in tqdm(dataset.images_list):
    preds= []
    
    for img_path in d:
        img = read_dataset_img(img_path, model.transform)
        preds.append(torch.nn.functional.softmax(model.forward(img), dim=1).detach().cpu().numpy())
        
    predictions.append(preds)
    
labels = dataset.labels

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
pred = []
for p in predictions:
    pred_proba = np.squeeze(np.array(p), axis=1)
    pred.append(np.argmax(pred_proba, axis=1))

## Camera Switch

In [13]:
np.squeeze(np.array(predictions[0][:5]), axis=1)[:, 0]

array([6.5123024e-12, 1.3130646e-11, 4.2012596e-12, 1.8361937e-11,
       4.8318355e-10], dtype=float32)

In [14]:
np.argpartition(np.squeeze(np.array(predictions[0][:5]), axis=1)[:, 1], -2)[-2:]

array([1, 4])

In [15]:
# def choose_two_best(valid_cams, cams_proba, num_cams=2):
#     if sum(valid_cams) == num_cams:
#         idx = np.where(valid_cams == True)
#         return idx[0], idx[1]
    
#     elif sum(valid_cams) > num_cams:
#         # find indexes of two maximal values in probabilities class 1
#         idx = np.argpartition(cam_proba[:, 1], -num_cams)[-num_cams:]
#         return idx[0], idx[1]
    
#     elif sum(valid_cams) < 2:
#         # find indexes of two minims values in probabilities class 0
#         idx = np.argpartition(cam_proba[:, 1], -2)[-2:]
#         return idx[0], idx[1]
        

def get_valid_cams(idx, pred, pred_proba, num_cams=2):
    valid_cams = [p[idx] for p in pred]
    cams_proba = np.squeeze([p[idx] for p in pred_proba], axis=1)
    
#     print(cams_proba)
    cam_ids = np.argpartition(cams_proba[:, 1], -2)[-2:]

    return cam_ids[0], cam_ids[1]

In [16]:
N = pred[0].shape[0]
cameras = []

for i in range(N):
    cam1, cam2 = get_valid_cams(i, pred, predictions)
    cameras.append([cam1, cam2])
    

In [17]:
dataset.features_list[cameras[0], 0]

TypeError: list indices must be integers or slices, not tuple

In [ ]:
cam1_data = []
cam2_data = []

for i in range(len(cameras)):
    cam1_data.append(dataset.features_list[cameras[i][0]][i])
    cam2_data.append(dataset.features_list[cameras[i][1]][i])

## Visualization

In [ ]:
from torchvision.io import read_image
import pandas as pd
from pathlib import Path
import os
import numpy as np

PYTHON_PATHS = ["../orb-classification/"]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

from data_loader.orb_features_dataset import ORBFeaturesDataset
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
# dataset_params = {'root': "/home/alena/Documents/omni-vision/dataset", 
#                   'datasets': [
#                               "dataset_2_cam_0",
#                               "dataset_2_cam_1",
#                               "dataset_2_cam_2",
#                              #"dataset_1_cam_0",
#                              ],
#                   'classification_threshold': 300,
#                   'transform': None,
#                   'combine_data': False
#                  }

# dataset = ORBFeaturesDataset(**dataset_params)

In [ ]:
for i in range(len(dataset.labels)):
    plt.subplots(figsize=(8, 2))
    plt.title(f"Number of ORB features on sequence{0}, camera{i}")
    plt.xlabel("frame num")
    plt.ylabel("num features")
    plt.plot(dataset.features_list[i])
    plt.show();

In [ ]:
running_mean_coeff = 20

def running_mean(x, N=running_mean_coeff):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

cam_features = []
for i in range(len(dataset.labels)):
    cam_features.append(running_mean(dataset.features_list[i]))

In [ ]:
for i in range(len(dataset.labels)):
    plt.subplots(figsize=(8, 2))
    plt.title(f"Number of ORB features on sequence{0}, camera{i}")
    plt.xlabel("frame num")
    plt.ylabel("num features")
    plt.plot(dataset.features_list[i])
    plt.plot(cam_features[i])
    plt.show();

    
cam2_data_s = running_mean(cam2_data)
plt.subplots(figsize=(8, 2))
plt.title(f"Our approach with camera switch")
plt.xlabel("frame num")
plt.ylabel("num features")
plt.plot(cam2_data)
plt.plot(cam2_data_s)
plt.show()

In [ ]:
plt.subplots(figsize=(16, 6))
plt.title(f"Number of ORB features on sequence{0}, camera{i}")
plt.xlabel("frame num")
plt.ylabel("num features")
plt.plot(cam_features[0])
plt.plot(cam_features[1])
plt.plot(cam_features[2])
plt.plot(cam2_data_s, linewidth=3, color='k')
plt.legend(["cam0", "cam1", "cam2", "ours"])
plt.show();

In [ ]:
c='rgbyk'
for i in range(len(dataset.labels)):
    plt.subplots(figsize=(8, 2))
    plt.title(f"Number of ORB features on sequence{0}, camera{i}")
    plt.xlabel("frame num")
    plt.ylabel("num features")
    plt.plot(cam_features[i], color=c[i])
    plt.show();

In [ ]:
for cam in cameras:
    

In [ ]:
import plotly.graph_objects as go

dataset_size = len(dataset.labels)
# x = np.linspace(0, dataset_size, dataset_size)
# y = dataset.features_list[0]

for i in range(dataset_size):
    fig = go.Figure()
    y = dataset.features_list[i][:500]
    x = np.linspace(0, y.shape[0], y.shape[0])
    
    fig.add_trace(go.Bar(x=x, y=y))

    fig.show()